In [11]:
#Imports
#Data processing
import pandas as pd
import numpy as np

#ML libs
from sklearn.impute import KNNImputer

In [4]:
ai_grid = np.load('../map_data/2d_ai.npy')

In [5]:
print(ai_grid)

[[7330745.12202422 7331760.53488938 7334060.40454262 ... 7497094.21929584
  7497790.23583703 7498039.48550125]
 [7333197.5232591  7336590.44627886 7340141.09219594 ... 7492364.35635126
  7493427.2356891  7491384.0372257 ]
 [7339554.15399274 7339822.34125162 7346433.56267525 ... 7490250.82347288
  7490639.03251509 7489313.08234804]
 ...
 [7339324.42321982 7338370.27392434 7337808.91653805 ... 7479945.20093714
  7480506.61698691 7480261.31217404]
 [7337509.51722756 7335284.93613764 7334043.33226672 ... 7482532.65559477
  7482650.45830715 7481127.24688628]
 [7334441.44000326 7334579.45702932 7332046.40679101 ... 7484917.88347116
  7484380.25223172 7484013.64450351]]


In [6]:
dfPrePrd = pd.read_csv(r'..\well_data\wellbore_data_preproduction_well.csv')

In [7]:
import os
print(os.getcwdb())

b'C:\\Users\\irvin\\PGE Hackathon\\MOAR-Drilling\\notebooks'


In [8]:
dfPrePrd.head()

,Well_ID,"X, m","Y, m","Depth, m","Porosity, fraction","Permeability, mD","Acoustic Impedance, kg*s/m^2",Rock facies,"Density, g/cm3","Compressible velocity, m/s","Youngs modulus, GPa","Shear velocity, m/s","Shear modulus, GPa"
0,Well_no_74,7925,5525,3050.51,NaN,NaN,7842692.077,Sandy shale,NaN,3387.576377,NaN,1646.603549,6.229941
1,Well_no_74,7925,5525,3051.01,0.09096,NaN,7807355.262,Shaly sandstone,1.872982,NaN,NaN,1465.051728,4.930174
2,Well_no_74,7925,5525,3051.51,0.05878,NaN,7864859.402,Sandy shale,3.187123,2660.306664,18.875428,1570.481116,7.202703
3,Well_no_74,7925,5525,3052.01,NaN,NaN,7309508.797,Shaly sandstone,2.058578,3285.615950,26.992396,1652.096739,6.486131
4,Well_no_74,7925,5525,3052.51,0.12285,NaN,7328552.658,Shaly sandstone,1.678339,4432.429019,35.261378,1697.320552,3.802001


In [17]:
dfProd = pd.read_csv((r"../well_data/wellbore_data_producer_wells.csv")).drop("Well_ID", axis=1)
dfProd.head()

,"X, m","Y, m","Depth, m","Porosity, fraction","Permeability, mD","Acoustic Impedance, kg*s/m^2",Rock facies,"Density, g/cm3","Compressible velocity, m/s","Youngs modulus, GPa","Shear velocity, m/s","Shear modulus, GPa"
0,7325,7175,3052.8,0.13330,NaN,6981171.853,Sandstone,2.280137,3106.544655,24.721555,1690.417133,7.304717
1,7325,7175,3053.3,0.13865,NaN,7234748.871,Sandstone,2.135061,4003.697087,23.360728,1573.847967,5.202120
2,7325,7175,3053.8,0.14638,NaN,7157383.755,Sandstone,1.991045,3462.569030,28.232152,1636.279139,NaN
3,7325,7175,3054.3,NaN,NaN,NaN,Sandstone,1.694242,3836.960702,29.220132,1613.043048,5.074763
4,7325,7175,3054.8,0.14993,NaN,NaN,Sandstone,1.664371,3919.585777,NaN,1636.846284,5.277834


In [15]:
imputer = KNNImputer(n_neighbors=3)

In [19]:
dfProd.describe()

,"X, m","Y, m","Depth, m","Porosity, fraction","Permeability, mD","Acoustic Impedance, kg*s/m^2","Density, g/cm3","Compressible velocity, m/s","Youngs modulus, GPa","Shear velocity, m/s","Shear modulus, GPa"
count,1460.000000,1460.000000,1460.000000,1268.000000,284.000000,1.357000e+03,1248.000000,1276.000000,1261.000000,1270.000000,1285.000000
mean,4188.698630,5770.890411,3055.810959,0.127746,112.182739,7.325786e+06,2.052521,3688.657853,27.288877,1676.158075,5.752640
std,1771.495039,2389.773641,4.353335,0.036020,80.894436,3.101125e+05,0.417622,727.216238,5.424608,100.630094,1.518699
min,1175.000000,775.000000,3045.620000,0.040270,2.214604,6.559277e+06,1.421943,1661.828496,13.205134,1307.887097,1.650985
25%,2925.000000,3825.000000,3052.720000,0.113953,51.540980,7.111056e+06,1.734052,3178.389486,23.466538,1608.928309,4.757475
50%,3875.000000,6125.000000,3055.790000,0.137495,93.980722,7.274333e+06,1.988542,3664.196826,26.647466,1674.836574,5.610267
75%,5325.000000,7625.000000,3058.982500,0.152325,154.584890,7.503378e+06,2.229507,4214.924966,30.874629,1746.081251,6.600503
max,7975.000000,9775.000000,3066.300000,0.188910,489.219402,8.249272e+06,3.530373,6179.653016,48.123292,1989.829247,11.820038


In [18]:
dfProd.isna().sum()

X, m                               0
Y, m                               0
Depth, m                           0
Porosity, fraction               192
Permeability, mD                1176
Acoustic Impedance, kg*s/m^2     103
Rock facies                      130
Density, g/cm3                   212
Compressible velocity, m/s       184
Youngs modulus, GPa              199
Shear velocity, m/s              190
Shear modulus, GPa               175
dtype: int64

In [16]:
dfProdimputed = imputer.fit_transform(dfProd)

ValueError: could not convert string to float: 'Sandstone'